In [ ]:
#Prediction part
def predict_subcategory_future_sales(data, sub_category, n_months):#goes into the prediction function
    # Step 1: Filter and aggregate
    df = data[data['Sub-Category'] == sub_category].copy()
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.groupby(pd.Grouper(key='Date', freq='M')).agg({'Quantity': 'sum', 'Profit': 'sum'}).reset_index()
    
    # Step 2: Feature engineering
    df['Month'] = df['Date'].dt.month
    df['Year'] = df['Date'].dt.year
    df['TimeIndex'] = range(len(df))
    
    # Step 3: Train on full data
    X = df[['TimeIndex', 'Month', 'Year']]
    y = df['Quantity']  
                
    model = RandomForestRegressor()
    model.fit(X, y)
    

    last_date = df['Date'].max()
    future_dates = [last_date + relativedelta(months=i+1) for i in range(n_months)]

    future_df = pd.DataFrame({'Date': future_dates,'Month': [d.month for d in future_dates],'Year': [d.year for d in future_dates],'TimeIndex': [df['TimeIndex'].max() + i + 1 for i in range(n_months)]})

    future_X = future_df[['TimeIndex', 'Month', 'Year']]

                
    future_df['Predicted Quantity'] = np.round(model.predict(future_X)).astype(int)
    plt.figure(figsize=(12,6))
    plt.plot(df['Date'], df['Quantity'], label='Historical Quantity')
    plt.plot(future_df['Date'], future_df['Predicted Quantity'], label='Predicted Future Quantity', linestyle='--')
    plt.legend()
    plt.xlabel('Date')
    plt.ylabel('Quantity')
    plt.title(f'Future Sales Prediction for {sub_category}')
    plt.grid(True)
    plt.show()
                

    return future_df